## Better identification of bus trips by ignoring Bus Time inferences
This notebook demonstrates that row-rise processing of time-sorted vehicle pings can be used to split the records according to unique bus trips.  The result of the splitting produces a total trip count that is much closer to the expected number (based on the schedule) for a sample date.

In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from itertools import compress
import time
%matplotlib inline  
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/bus-Capstone')

# these two modules are homemade
import gtfs
import arrivals
import time
import ttools
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

### Get raw data

In [4]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('20151203_parsed_with_destinations.csv')       
bustime.rename(columns={'vehicleID':'vehicle_id','Line':'route','Latitude':'lat','Longitude':'lon',
                        'Trip':'trip_id','TripDate':'trip_date','TripPattern':'shape_id',
                        'MonitoredCallRef':'next_stop_id','DistFromCall':'dist_from_stop',
                        'CallDistAlongRoute':'stop_dist_on_trip','RecordedAtTime':'timestamp'},inplace=True)

In [9]:
bustime.drop_duplicates(['vehicle_id','timestamp'],inplace=True)
bustime['trip_id'] = bustime['trip_id'].str.replace('MTA NYCT_','')
bustime['trip_id'] = bustime['trip_id'].str.replace('MTABC_','')
bustime['ts'] = bustime['timestamp'].apply(ttools.parseActualTime,tdate='2015-12-03')
print 'Loaded Bus Time data but did not set indexes'

Loaded Bus Time data but did not set indexes


In [10]:
# for demonstration, use a subset. Just get data one day.
tripDateLookup = "2015-12-03"
bustime = bustime[bustime['trip_date']==tripDateLookup]
# note that the AVL dataframe must be sorted by timestammp, since iloc[]
# selection is used later in this script to find the earliest time


bustime['stop_dist_on_trip'] = bustime['stop_dist_on_trip'].convert_objects(convert_numeric=True)
bustime['dist_from_stop'] = bustime['dist_from_stop'].convert_objects(convert_numeric=True)
bustime['veh_dist_along_trip'] = bustime['stop_dist_on_trip'] - bustime['dist_from_stop']

print 'Finished loading BusTime data, converting distances, and slicing ONE DAY.'

/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Finished loading BusTime data, converting distances, and slicing ONE DAY.


/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
bustime.set_index(['route','trip_date','vehicle_id'],inplace=True,drop=True)
bustime.set_index('timestamp',append=True,drop=True,inplace=True)
bustime.sort_index(inplace=True)

/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [14]:
import numpy as np

### Labeling algorithm
This algorithm interates row-wise through a dataframe and increments the label if any of two conditions is met:
   1. The headsign (destination displayed to customers) changes
   2. More than 45 minutes elapses since the bus last reported observations on the same route

In [59]:
def matthew4(df,thresh=45):
    label = 0
    first_idx = df.index[0]
    time_limit = ttools.datetime.timedelta(minutes=thresh)
    limit_bools = df['ts'].diff()>time_limit
    for index, row in df.iterrows():
        if index==first_idx:
            labels = [label]
            hs = row.DestinationRef
            ts = row.ts
        else:
            if (row.DestinationRef!=hs or limit_bools[index]==True):
                label += 1
                labels.append(label)
                hs = row.DestinationRef
                ts = row.ts
            else:
                labels.append(label)
    return pd.Series(data=labels,index=df.index.get_level_values(3))

In [63]:
bustime['new_trip_id'] = bustime.groupby(level=(0,1,2)).apply(matthew4)

### Compare length of label lists, using various groupings
Using a combination of the route, vehicle and new inferred trip_id label:

In [68]:
sum(bustime.set_index('new_trip_id',append=True).groupby(level=(0,1,2,4)).size()>2)

53575

Also adding the shape_id to the grouping (to show how often multiple shape_id are reported for the same trip).

In [77]:
sum(bustime.set_index(['new_trip_id','shape_id'],append=True).groupby(level=(0,1,2,4,5)).size()>2)

54389

### Examine one specific example of data where the shape_id and trip_id seem to change incorrectly.
Notice how trip_id and shape_id changed several times, even though the bus was going towards the same destination during the whole time span.

In [90]:
bustime.set_index(['new_trip_id','shape_id'],append=True).groupby(level=(0,1,2,4,5)).size()

route        trip_date   vehicle_id     new_trip_id  shape_id   
MTA NYCT_B1  2015-12-03  MTA NYCT_4855  0            MTA_B1O0251     3
                                                     MTA_B1O0252    28
                                                     MTA_B1O0254    22
                                                     MTA_B1O0256     1
                                        1            MTA_B1O0243    36
                                        2            MTA_B1O0251    40
                                        3            MTA_B1O0243    33
                                        4            MTA_B1O0251    44
                                        5            MTA_B1O0244    15
                                        6            MTA_B1O0248     5
                                        7            MTA_B1O0251     1
                                        8            MTA_B1O0243    20
                                        9            MTA_B1O0244     1
            

In [89]:
example_idx = bustime.set_index(['new_trip_id','shape_id'],append=True).groupby(level=(0,1,2,4,5)).size().index[0]
bustime.set_index(['new_trip_id','shape_id'],append=True).xs(example_idx[:-1],level=(0,1,2,4))

,,lat,lon,trip_id,DestinationRef,DestinationName,next_stop_id,EstCallArrival,dist_from_stop,stop_dist_on_trip,PresentableDistance,ResponseTimeStamp,ts,veh_dist_along_trip
timestamp,shape_id,,,,,,,,,,,,,
2015-12-03T06:36:22.000-05:00,MTA_B1O0251,40.607162,-74.002487,UP_D5-Weekday-SDon-037300_B1_7,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_300099,2015-12-03T06:36:29.289-05:00,19.46,2919.71,at stop,2015-12-03T06:36:16.377-05:00,06:36:22,2900.25
2015-12-03T06:37:26.000-05:00,MTA_B1O0251,40.607144,-74.002457,UP_D5-Weekday-SDon-037300_B1_7,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_300099,2015-12-03T06:37:36.878-05:00,16.22,2919.71,at stop,2015-12-03T06:37:35.878-05:00,06:37:26,2903.49
2015-12-03T06:39:02.000-05:00,MTA_B1O0252,40.606077,-74.000695,UP_D5-Weekday-SDon-039800_B1_10,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_300100,2015-12-03T06:39:20.567-05:00,13.65,188.13,at stop,2015-12-03T06:39:19.567-05:00,06:39:02,174.48
2015-12-03T06:40:36.000-05:00,MTA_B1O0252,40.603497,-73.996420,UP_D5-Weekday-SDon-039800_B1_10,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_300102,2015-12-03T06:40:43.118-05:00,23.60,659.15,at stop,2015-12-03T06:40:36.973-05:00,06:40:36,635.55
2015-12-03T06:42:11.000-05:00,MTA_B1O0252,40.602113,-73.994078,UP_D5-Weekday-SDon-039800_B1_10,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_300103,2015-12-03T06:42:28.232-05:00,13.76,899.91,at stop,2015-12-03T06:42:27.232-05:00,06:42:11,886.15
2015-12-03T06:43:46.000-05:00,MTA_B1O0252,40.600725,-73.991835,UP_D5-Weekday-SDon-039800_B1_10,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_300104,2015-12-03T06:43:57.756-05:00,13.56,1143.96,at stop,2015-12-03T06:43:56.756-05:00,06:43:46,1130.40
2015-12-03T06:45:21.000-05:00,MTA_B1O0252,40.598614,-73.988345,UP_D5-Weekday-SDon-039800_B1_10,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_300106,2015-12-03T06:45:49.235-05:00,112.45,1619.57,approaching,2015-12-03T06:45:30.428-05:00,06:45:21,1507.12
2015-12-03T06:46:24.000-05:00,MTA_B1O0252,40.596946,-73.985593,UP_D5-Weekday-SDon-039800_B1_10,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_306457,2015-12-03T06:47:05.785-05:00,34.23,1838.64,approaching,2015-12-03T06:46:55.609-05:00,06:46:24,1804.41
2015-12-03T06:48:32.000-05:00,MTA_B1O0252,40.594945,-73.982277,UP_D5-Weekday-SDon-039800_B1_10,MTA_901472,MNHTN BCH KNGSBRO CC via 13 AV via 86 ST,MTA_300109,2015-12-03T06:49:25.443-05:00,206.81,2368.86,< 1 stop away,2015-12-03T06:48:52.232-05:00,06:48:32,2162.05


And compare to using the old method, which is to group by combination of vehicle and reported trip_id only.

In [69]:
bt_copy = bustime.set_index('trip_id',append=True)

In [71]:
sum(bt_copy.groupby(level=(0,1,2,4)).size()>2)

57232

In [72]:
sum(bt_copy.groupby(level=(0,1,4)).size()>2)

51102

## Now try a different algorithm
Instead of using elapsed time as a way to detect a break, check if the vehicle moves more than a certain distance "backwards" since the last ping associated with that headsign.

In [93]:
bustime['DestinationRef'] = bustime['DestinationRef'].str.replace('MTA_','')

In [100]:
bustime['DestinationRef'] = bustime['DestinationRef'].convert_objects(convert_numeric=True)

In [96]:
stops = gtfs.load_stops('2015-12-03','gtfs/')

,location_type,parent_station,stop_desc,stop_lat,stop_lon,stop_name,stop_url,zone_id
stop_id,,,,,,,,
300000,0,NaN,NaN,40.578251,-73.939743,ORIENTAL BL/MACKENZIE ST,NaN,NaN
300002,0,NaN,NaN,40.578068,-73.943031,ORIENTAL BL/JAFFRAY ST,NaN,NaN
300003,0,NaN,NaN,40.577961,-73.944664,ORIENTAL BL/HASTINGS ST,NaN,NaN
300004,0,NaN,NaN,40.577721,-73.946205,ORIENTAL BL/FALMOUTH ST,NaN,NaN
300006,0,NaN,NaN,40.577354,-73.949554,ORIENTAL BL/DOVER ST,NaN,NaN


In [115]:
bustime = bustime.merge(stops[['stop_lat','stop_lon']],left_on='DestinationRef',right_index=True,how='left').rename(columns={'stop_lat':'dest_lat','stop_lon':'dest_lon'})

In [107]:
import numpy as np
def dest_dist_eucl(row):
    a = (row.dest_lat - row.lat)
    b = (row.dest_lon - row.lon)
    return np.sqrt((a**2 + b**2))

In [116]:
bustime['dest_eucl']= bustime.apply(dest_dist_eucl,axis=1)

In [110]:
def matthew5(df,thresh=0.01):
    label = 0
    first_idx = df.index[0]
    limit_bools = df['dest_eucl'].diff()<(-1*thresh)
    for index, row in df.iterrows():
        if index==first_idx:
            labels = [label]
            hs = row.DestinationRef
            ts = row.ts
        else:
            if (row.DestinationRef!=hs or limit_bools[index]==True):
                label += 1
                labels.append(label)
                hs = row.DestinationRef
                ts = row.ts
            else:
                labels.append(label)
    return pd.Series(data=labels,index=df.index.get_level_values(3))

In [117]:
bustime['new_trip_id2'] = bustime.groupby(level=(0,1,2)).apply(matthew5)

In [118]:
sum(bustime.set_index('new_trip_id2',append=True).groupby(level=(0,1,2,4)).size()>2)

72460